In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os, tqdm, sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.save import make_dir, save_pickle, load_pickle, save
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from collections import Counter
import spacy
import copy
import re
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [2]:
dic = load(dir_save = '../big_data/dic_20190927.pickle')

exist ../big_data/dic_20190927.pickle
time: 3.55 s


In [3]:
### STEP2 load and clean the generation

def reverse(text):
    '''
    Important data cleaning before NY times parser
    '''
    # replace things in brace
    text = re.sub(r'\([^)]*\)', '', text)

    # remove space before punct
    text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', text)

    # remove consecutive spaces
    text = re.sub(' +',' ',text).strip()
    return text

def reverse_list(listoftext):
    output=[]
    for text in listoftext:
        rev = reverse(text)
        if rev:
            output.append(rev)
    return output

def load_dir_data(filename):
    ls = []
    if os.path.isdir(filename):
        print('load', filename)
        # Directory
        for (dirpath, _, fnames) in os.walk(filename):
            for fname in fnames:
                path = os.path.join(dirpath, fname)
                with open(path, 'r') as fp:
                    raw_text = fp.read()
                    
                # if it contains instr
                if fname[-5] == 'd':
                    dic[int(fname[:-5])]['generated_instr'] = reverse_list(raw_text.split('.'))

                # if it contains ingred
                if fname[-5] == 'i':
                    dic[int(fname[:-5])]['generated_ingred'] = reverse_list(raw_text.split('$'))
                    
                # if it contains name
                if fname[-5] == 't':
                    dic[int(fname[:-5])]['generated_name'] = raw_text
                ls.append(int(fname[:-5]))# only interested in instr
                    
    return sorted(list(set(ls)))

time: 24.8 ms


In [4]:
filename = '../../to_gpt2/generation_20191008-scratch/'
ls = load_dir_data(filename)
#filename = '../../to_gpt2/generation_1221k_top0.95_new/'
#ls = load_dir_data(filename)
assert len(ls)>0

load ../../to_gpt2/generation_20191008-scratch/
time: 225 ms


In [6]:
def add_space(line):
    # add space before punct
    line = re.sub('([.,!?()])', r' \1 ', line)
    line = re.sub('\s{2,}', ' ', line)
    return line

to_write = {'truth_t':'', 'truth_i':'', 'truth_d':'',
            'pred_t':'', 'pred_i':'', 'pred_d':''
           }
for i, v in dic.items():
    if i in ls:
        to_write['truth_t'] += add_space(' '.join(v['name'])) + '\n'
        to_write['truth_i'] += add_space(' $ '.join(v['ingredients']))+ ' $ \n'
        to_write['truth_d'] += add_space(' '.join(v['directions'])) + ' . \n'
        to_write['pred_t'] += add_space(v['generated_name']) + '\n'
        to_write['pred_i'] += add_space(' $ '.join(v['generated_ingred'])) + ' $ \n'
        to_write['pred_d'] += add_space(' . '.join(v['generated_instr'])) + ' . \n'
        
for k, v in to_write.items():
    save('../../to_gpt2/generation_20191008_inv_%s.txt'%(k), v ,overwrite = True)

saved ../../to_gpt2/generation_20191008_inv_truth_t.txt
saved ../../to_gpt2/generation_20191008_inv_truth_i.txt
saved ../../to_gpt2/generation_20191008_inv_truth_d.txt
saved ../../to_gpt2/generation_20191008_inv_pred_t.txt
saved ../../to_gpt2/generation_20191008_inv_pred_i.txt
saved ../../to_gpt2/generation_20191008_inv_pred_d.txt
time: 477 ms


In [7]:
!perl multi-bleu.perl ../../to_gpt2/generation_20191008_inv_truth_t.txt < ../../to_gpt2/generation_20191008_inv_pred_t.txt

BLEU = 3.41, 27.7/7.6/1.9/0.4 (BP=0.980, ratio=0.980, hyp_len=1945, ref_len=1985)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
time: 186 ms


In [8]:
!perl multi-bleu.perl ../../to_gpt2/generation_20191008_inv_truth_i.txt < ../../to_gpt2/generation_20191008_inv_pred_i.txt

BLEU = 23.23, 53.9/32.1/18.3/9.2 (BP=1.000, ratio=1.108, hyp_len=29944, ref_len=27015)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
time: 519 ms


In [9]:
!perl multi-bleu.perl ../../to_gpt2/generation_20191008_inv_truth_d.txt < ../../to_gpt2/generation_20191008_inv_pred_d.txt

BLEU = 12.72, 54.1/21.5/9.4/4.8 (BP=0.840, ratio=0.852, hyp_len=51324, ref_len=60267)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
time: 910 ms
